How many citations (incoming and outgoing), as in OpenCitations, are the research objects involved in?

In [ ]:
import requests
import pandas as pd
import time

df= pd.read_csv("C:/Users/annan/Downloads/mashup.csv", low_memory=False)
list_doi= df["doi"].dropna().tolist() 

In [1]:
#access API to retrieve outgoing citations
def get_references(doi):        
    url = f"https://opencitations.net/index/api/v2/references/doi:{doi}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching references for {doi}: {response.status_code}")
        return []


In [2]:
#access API to retrieve ingoing citations
def get_citations(doi):     
    url = f"https://opencitations.net/index/api/v2/citations/doi:{doi}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching citations for {doi}: {response.status_code}")
        return []


In [3]:
#creating DF for outgoing cit
rows1 = []    

for doi in list_doi:
    outgoing_cit= list()
    outgoing_oci = set() 
    citations = get_references(doi)
    for cit in citations:   #check duplicated citation through oci
        oci = cit.get("oci")
        if oci is not None and oci not in outgoing_oci:
            outgoing_cit.append(cit)
            outgoing_oci.add(oci)
    rows1.append({
        'doi': doi,
        'cite_num_outgoing': len(outgoing_cit),
        'oci_outgoing': list(outgoing_oci)
        })
    time.sleep(1)

output_table1 = pd.DataFrame(rows1)


NameError: name 'list_doi' is not defined

In [4]:
#creating DF for ingoing cit
rows2 = []   

for doi in list_doi:
    ingoing_cit= list()
    ingoing_oci = set() 
    citations = get_citations(doi)
    for cit in citations:
        oci = cit.get("oci")
        if oci is not None and oci not in ingoing_oci:
            ingoing_cit.append(cit)
            ingoing_oci.add(oci)
    rows2.append({
        'doi': doi,
        'cite_num_ingoing': len(ingoing_cit),
        'oci_ingoing': list(ingoing_oci)
        })
    time.sleep(1)

output_table2 = pd.DataFrame(rows2)


NameError: name 'list_doi' is not defined

In [ ]:
#merge to a single DF and export csv
from IPython.display import FileLink

output = pd.merge(output_table1, output_table2, on='doi', how="outer")
output.to_csv("CitationsCount", index=False)
Filelink("CitationsCount")